In [43]:
using CSV, DataFrames
using LinearAlgebra, Random, Statistics
using Convex, SCS

In [44]:
fname = "datasets/zcta_dataset.csv"
df = DataFrame(CSV.File(fname));

In [45]:
cols = names(df)
for col in cols
    m = .!isnan.(df[:,col])
    println("$(col) $(sum(.!m))")
end

ZIP 0
single_parent_household 2397
youth_not_in_school 3799
sex_pay_inequity 840
race_pay_inequity 21691
snap_household 944
snap_vulnerabe_household 944
no_internet_household 944
no_smartphone_household 944
no_car_household 944
no_plumbing_household 944
over_60min_commute 1017
biking_commute_inv 958
working_class_workers 973
unemployment_rate 0
renter_occupied 0
median_income_inv 0
Crimes Against Persons Rate 11165
Crimes Against Property Rate 11165
Crimes Against Society Rate 11165
ALR_VALB 2128
ALR_VALP 2128
lahunv1share 4789
lasnap1share 4741


In [46]:
(n, m) = size(df[!, cols[2:end]])

(33772, 23)

In [47]:
A = Matrix(df[:, cols[2:end]])
M = isnan.(A)

corr = zeros(m,m)
for i in 1:m
    for j in i:m
        mask = (M[:,i] .+ M[:,j]) .== 0
        x = cor(Vector(df[mask ,cols[1+i]]), Vector(df[mask ,cols[1+j]]))
        corr[i, j] = x
        corr[j, i] = x
    end
end

eigs = eigvals(corr)
nfa = sum(eigs .> 1)
k = nfa

7

In [34]:
df_Y = DataFrame(CSV.File("weights/county_Y.csv"))
k_init = minimum([size(df_Y)[1], k])
Y_prev = Matrix(df_Y)[1:k_init, :];

In [35]:
MAX_ITERS = 20
residual = zeros(MAX_ITERS);
rng = Xoshiro(42)

X_0 = rand(rng, Float64,(n, k));
X_0 = X_0 ./ (X_0 * ones(k, 1))

Y_0 = rand(rng, Float64,(k, m));
Y_0[1:k_init, :] = Y_prev
H = ones((n, m))
p = 0
constraint = []

A[M] .= 0
H[M] .= 0

step = 2500

for i = 1:MAX_ITERS
    
    if (i % 2) == 0
        Y = Variable((k,m))
        set_value!(Y, Y_0)
        X = X_0
        #constraint = [Y >= 0]
        constraint = [Y[1,:] >= 0]
        #constraint = []
        obj = sum(huber(A - X*Y.*H, .4)) + 12 * norm(Y[2:k,:], 1) + 6 * norm(Y[2:k,:], 2)

        p = minimize(obj, constraint)
        solve!(p, SCS.Optimizer; silent = false)

        if p.status != Convex.MOI.OPTIMAL
            println("Failed to Converge")
        end

        Y_0 = Y.value

        residual[i] = p.optval
        println("$i, $(residual[i])")

    else
        for j in 1:step:n
            if j + step > n
                dim = n - j + 1
            else
                dim = step
            end
                
            X = Variable((dim, k))
            set_value!(X, X_0[j:j+dim-1, :])
            Y = Y_0
            #constraint = [X >= 0, X * ones(k, 1) <= 1]
            constraint = [X[:,1] >= 0, opnorm(X, Inf) <= 1]
            #constraint = []
            obj = sum(huber(A[j:j+dim-1, :] - X*Y.*H[j:j+dim-1, :], .4))

            p = minimize(obj, constraint)
            solve!(p, SCS.Optimizer; silent = false)

            if p.status != Convex.MOI.OPTIMAL
                println("Failed to Converge")
            end

            residual[i] += p.optval
            
            X_0[j:j+dim-1, :] = X.value
        end
         println("$i, $(residual[i])")
    end
end

println()

w = mean(abs.(X_0), dims=1) .* mean(abs.(Y_0), dims=2)'
w = w ./ sum(w)
println(w)

[ Info: [Convex.jl] Compilation finished: 0.53 seconds, 346.586 MiB of memory allocated


------------------------------------------------------------------
	       SCS v3.2.7 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
------------------------------------------------------------------
problem:  variables n: 275001, constraints m: 400001
cones: 	  z: primal zero / dual free vars: 60000
	  l: linear vars: 160001
	  q: soc vars: 180000, qsize: 60000
settings: eps_abs: 1.0e-004, eps_rel: 1.0e-004, eps_infeas: 1.0e-007
	  alpha: 1.50, scale: 1.00e-001, adaptive_scale: 1
	  max_iters: 100000, normalize: 1, rho_x: 1.00e-006
	  acceleration_lookback: 10, acceleration_interval: 10
	  compiled with openmp parallelization enabled
lin-sys:  sparse-direct-amd-qdldl
	  nnz(A): 1006903, nnz(P): 0
------------------------------------------------------------------
 iter | pri res | dua res |   gap   |   obj   |  scale  | time (s)
------------------------------------------------------------------
     0|1.45e+001 1.00e+000 1.74e+006 -8.71e+005 1.00e-001 6.17e

[ Info: [Convex.jl] Compilation finished: 0.72 seconds, 344.036 MiB of memory allocated


------------------------------------------------------------------
	       SCS v3.2.7 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
------------------------------------------------------------------
problem:  variables n: 275001, constraints m: 400001
cones: 	  z: primal zero / dual free vars: 60000
	  l: linear vars: 160001
	  q: soc vars: 180000, qsize: 60000
settings: eps_abs: 1.0e-004, eps_rel: 1.0e-004, eps_infeas: 1.0e-007
	  alpha: 1.50, scale: 1.00e-001, adaptive_scale: 1
	  max_iters: 100000, normalize: 1, rho_x: 1.00e-006
	  acceleration_lookback: 10, acceleration_interval: 10
	  compiled with openmp parallelization enabled
lin-sys:  sparse-direct-amd-qdldl
	  nnz(A): 986435, nnz(P): 0
------------------------------------------------------------------
 iter | pri res | dua res |   gap   |   obj   |  scale  | time (s)
------------------------------------------------------------------
     0|1.45e+001 1.00e+000 1.74e+006 -8.71e+005 1.00e-001 6.11e-

[ Info: [Convex.jl] Compilation finished: 0.68 seconds, 344.705 MiB of memory allocated


...

------------------------------------------------------------------
	       SCS v3.2.7 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
------------------------------------------------------------------
problem:  variables n: 275001, constraints m: 400001
cones: 	  z: primal zero / dual free vars: 60000
	  l: linear vars: 160001
	  q: soc vars: 180000, qsize: 60000
settings: eps_abs: 1.0e-004, eps_rel: 1.0e-004, eps_infeas: 1.0e-007
	  alpha: 1.50, scale: 1.00e-001, adaptive_scale: 1
	  max_iters: 100000, normalize: 1, rho_x: 1.00e-006
	  acceleration_lookback: 10, acceleration_interval: 10
	  compiled with openmp parallelization enabled
lin-sys:  sparse-direct-amd-qdldl
	  nnz(A): 993757, nnz(P): 0
------------------------------------------------------------------
 iter | pri res | dua res |   gap   |   obj   |  scale  | time (s)
------------------------------------------------------------------
     0|1.45e+001 1.00e+000 1.74e+006 -8.71e+005 1.00e-001 6.10e-

[ Info: [Convex.jl] Compilation finished: 0.19 seconds, 181.093 MiB of memory allocated


------------------------------------------------------------------
	       SCS v3.2.7 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
------------------------------------------------------------------
problem:  variables n: 139921, constraints m: 203521
cones: 	  z: primal zero / dual free vars: 30528
	  l: linear vars: 81409
	  q: soc vars: 91584, qsize: 30528
settings: eps_abs: 1.0e-004, eps_rel: 1.0e-004, eps_infeas: 1.0e-007
	  alpha: 1.50, scale: 1.00e-001, adaptive_scale: 1
	  max_iters: 100000, normalize: 1, rho_x: 1.00e-006
	  acceleration_lookback: 10, acceleration_interval: 10
	  compiled with openmp parallelization enabled
lin-sys:  sparse-direct-amd-qdldl
	  nnz(A): 506946, nnz(P): 0
------------------------------------------------------------------
 iter | pri res | dua res |   gap   |   obj   |  scale  | time (s)
------------------------------------------------------------------
     0|1.45e+001 1.00e+000 8.87e+005 -4.43e+005 1.00e-001 3.24e-00

[ Info: [Convex.jl] Compilation finished: 8.11 seconds, 4.458 GiB of memory allocated


------------------------------------------------------------------
	       SCS v3.2.7 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
------------------------------------------------------------------
problem:  variables n: 3242425, constraints m: 4863625
cones: 	  z: primal zero / dual free vars: 810528
	  l: linear vars: 1621368
	  q: soc vars: 2431729, qsize: 810529
settings: eps_abs: 1.0e-004, eps_rel: 1.0e-004, eps_infeas: 1.0e-007
	  alpha: 1.50, scale: 1.00e-001, adaptive_scale: 1
	  max_iters: 100000, normalize: 1, rho_x: 1.00e-006
	  acceleration_lookback: 10, acceleration_interval: 10
	  compiled with openmp parallelization enabled
lin-sys:  sparse-direct-amd-qdldl
	  nnz(A): 12370147, nnz(P): 0
------------------------------------------------------------------
 iter | pri res | dua res |   gap   |   obj   |  scale  | time (s)
------------------------------------------------------------------
     0|1.79e+002 1.20e+001 2.38e+007 -1.19e+007 1.00e-00

In [38]:
for i = 1:20
    println("$i, $(residual[i])")
end

1, 33374.09771338302
2, 29515.490100255043
3, 26361.60964829682
4, 26089.16218177023
5, 24318.099002937404
6, 24971.751341607924
7, 23497.36613431383
8, 24471.96019872199
9, 23072.552343891806
10, 24192.87155666166
11, 22810.37984742625
12, 24013.318549954416
13, 22630.779941114157
14, 23887.975143080308
15, 22501.005211261094
16, 23797.445454072033
17, 22405.32621092551
18, 23731.196792448423
19, 22334.332618741133
20, 23682.134248912014


In [39]:
w = mean(abs.(X_0), dims=1) .* mean(abs.(Y_0), dims=2)'
w = w ./ sum(w)
println(w)

[0.46198606738435505 0.07256195254064932 0.11198909608719274 0.1046300061273567 0.09009424014999284 0.08089540549409342 0.07784323221635968]


In [41]:
df_Y = DataFrame(Y_0, cols[2:end])
fname = "weights/zcta_Y.csv"
CSV.write(fname, df_Y)

"weights/zcta_Y.csv"

In [42]:
df_X = df[:, ["ZIP"]]
for i in 1:k
    df_X[:, "$i"] = X_0[:, i]
end
fname = "weights/zcta_X.csv"
CSV.write(fname, df_X)

"weights/zcta_X.csv"

In [40]:
df

Row,ZIP,single_parent_household,youth_not_in_school,sex_pay_inequity,race_pay_inequity,snap_household,snap_vulnerabe_household,no_internet_household,no_smartphone_household,no_car_household,no_plumbing_household,over_60min_commute,wfh_inv,biking_commute_inv,working_class_workers,unemployment_rate,renter_occupied,median_income_inv,Crimes Against Persons Rate,Crimes Against Property Rate,Crimes Against Society Rate,ALR_VALB,ALR_VALP,lahunv1share,lasnap1share
,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,601,0.896249,0.783827,0.730899,0.0365096,0.992735,0.987906,0.854097,0.892191,0.908575,0.78517,0.670805,0.757227,1.0,0.853002,0.983137,0.709142,0.918047,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,602,0.87678,0.781514,0.765112,0.267241,0.992461,0.988058,0.904676,0.983905,0.874184,0.731219,0.164047,0.452961,0.702623,0.840726,0.735196,0.581503,0.855922,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,603,0.940194,0.65995,0.601565,0.181224,0.989408,0.981569,0.816102,0.871881,0.941135,0.725766,0.583293,0.74405,0.590536,0.653058,0.96897,0.861943,0.72415,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,606,0.905935,0.0,0.736852,NaN,0.992125,0.98952,0.994686,0.996732,0.877266,0.815116,0.56831,0.944193,1.0,0.982914,0.0279474,0.5398,0.884131,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,610,0.890919,0.713323,0.440074,0.150592,0.991667,0.985895,0.817446,0.980515,0.841257,0.738987,0.508592,0.811604,0.498974,0.912663,0.750845,0.645018,0.831508,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,611,0.798089,NaN,0.817459,NaN,0.993468,0.990495,0.942946,0.829307,0.884162,0.0,0.969453,0.595322,1.0,0.882833,0.97469,0.789198,0.88512,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,612,0.915069,0.539325,0.467199,0.167729,0.989133,0.97965,0.857793,0.881776,0.919408,0.705721,0.662457,0.76356,0.839111,0.741394,0.964821,0.825266,0.838898,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,616,0.94884,0.0,0.221479,NaN,0.990507,0.985011,0.881891,0.911737,0.83015,0.778621,0.583293,0.927168,1.0,0.847598,0.977654,0.702927,0.876476,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,617,0.878567,0.66166,0.105339,NaN,0.992979,0.983762,0.858954,0.826711,0.839579,0.640285,0.791341,0.824815,1.0,0.737021,0.95839,0.675602,0.792313,NaN,NaN,NaN,NaN,NaN,NaN,NaN
